# a first simplified 2D barotropic model
Use FFT in periodic 2D domain

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.fft import fft2, ifft2
from scipy.misc import derivative
import math as mt

%load_ext autoreload
%autoreload 1
%matplotlib inline
#%matplotlib notebook

def plot_model(psi, zeta, timestep):
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(2,2,1)
    h1=ax.contourf(np.real(psi))
    ax.set_title(f'Stream function at time step {timestep}')
    fig.colorbar(h1)
    ax = fig.add_subplot(2,2,2)
    h2=ax.contourf(np.real(zeta))
    ax.set_title(f'Vorticity at time step {timestep}')
    fig.colorbar(h2)
    plt.show()
    return

N = 96
T = 10001
deltaT = 0.0001
Tplot = 500
beta = 0.1

rob_filter_weigh_small = 0.03
rob_filter_weigh_big = 1 - rob_filter_weigh_small

x = np.linspace(-2*mt.pi, 2*mt.pi, N)
y = np.linspace(-2*mt.pi, 2*mt.pi, N)

kx = 2*mt.pi*np.fft.fftfreq(len(x), (x[1]-x[0])/2*mt.pi)
ky = 2*mt.pi*np.fft.fftfreq(len(x), (y[1]-y[0])/2*mt.pi)

x, y = np.meshgrid(x, y)

kx[0]=ky[0]=0
KX, KY = np.meshgrid(kx/2*mt.pi, ky/2*mt.pi,  indexing='xy')

kx[0]=ky[0]=1
KXu, KYu = np.meshgrid(kx/2*mt.pi, ky/2*mt.pi,  indexing='xy')

print(KYu)

def multivariate_gaussian(pos, mu, sigma):
    """Return the multivariate Gaussian distribution on array pos."""

    n = mu.shape[0]
    sigma_det = np.linalg.det(sigma)
    sigma_inv = np.linalg.inv(sigma)
    N = np.sqrt((2*np.pi)**n * sigma_det)
    # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
    # way across all the input variables.
    fac = np.einsum('...k,kl,...l->...', pos-mu, sigma_inv, pos-mu)

    return np.exp(-fac / 2) / N

In [ ]:
# initialization

# Mean vector and covariance matrix
mu = np.array([2., 1.])
sigma = np.array([[ 4. , -1.5], [-1.5,  4.]])

# Pack x and y into a single 3-dimensional array
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x
pos[:, :, 1] = y

# Mean vector and covariance matrix
mu1 = np.array([-4., -4.])
sigma1 = np.array([[ 4. , -1.5], [-1.5,  4.]])

# Pack x and y into a single 3-dimensional array
pos1 = np.empty(x.shape + (2,))
pos1[:, :, 0] = x
pos1[:, :, 1] = y

psi = multivariate_gaussian(pos, mu, sigma) - multivariate_gaussian(pos1, mu1, sigma1)
#print(psi)

plt.contourf(x,y,np.real(psi))

A = np.zeros((N,N), dtype=complex)
dAdx_fft = np.zeros((N,N), dtype=complex)
B = np.zeros((N,N), dtype=complex)
dBdy_fft = np.zeros((N,N), dtype=complex)

zeta_fft = np.zeros((N,N), dtype=complex)
u_fft = np.zeros((N,N), dtype=complex)
v_fft = np.zeros((N,N), dtype=complex)

In [ ]:
%%time

# Use mesh for vectorization

for t in np.arange(0,T):
    #print(f'time step {t}')
    psi_fft = fft2(psi)
    
    #for k in np.arange(0,N):
    #    for l in np.arange(0,N):
    #        zeta_fft[k,l] = (-k**2 - l**2)*psi_fft[k,l]
    #        u_fft[k,l] = -1j*l*psi_fft[k,l]
    #        v_fft[k,l] = 1j*k*psi_fft[k,l]
    zeta_fft = (-KX**2 - KY**2)*psi_fft
    u_fft = -1j*KX*psi_fft
    v_fft = 1j*KY*psi_fft
    
    zeta = ifft2(zeta_fft)
    u = ifft2(u_fft)
    v = ifft2(v_fft)
    # for i in np.arange(0,N):
    #     for j in np.arange(0,N):
    #         A[i,j] = -u[i,j]*zeta[i,j]
    #         B[i,j] = v[i,j]*zeta[i,j]
    A = -u*zeta
    B = v*zeta
    
    A_fft = fft2(A)
    B_fft = fft2(B)
    #for k in np.arange(0,N):
    #    for l in np.arange(0,N):
    #        dAdx_fft[k,l] = 1j*k*A_fft[k,l]
    #        dBdy_fft[k,l] = 1j*l*B_fft[k,l]
    dAdx_fft = 1j*KX*A_fft
    dBdy_fft = 1j*KY*B_fft
    
    dAdx = ifft2(dAdx_fft)
    dBdy = ifft2(dBdy_fft)
    #print(dAdx)
    
    f = -dAdx - dBdy #- psi*beta
    #print(f)
    
    # integration
    if t == 0:
        # forward scheme
        previous_zeta = zeta.copy()
        zeta += deltaT*f
    else:
        # leap-frog
        previous_zeta = zeta.copy()
        unfiltered_zeta = previous_zeta + 2*deltaT*f
    
        # Robert filter
        zeta = rob_filter_weigh_small*zeta + rob_filter_weigh_big*unfiltered_zeta
    
    zeta_fft = fft2(zeta)
    
    #for k in np.arange(0,N):
    #    for l in np.arange(0,N):
    #        if k == 0 and l == 0:
    #            psi_fft[k,l] = 0.0
    #        else:
    #            psi_fft[k,l] = zeta_fft[k,l]/(-k**2-l**2)
        
    psi_fft = zeta_fft/(-KXu**2 - KYu**2)
    #psi_fft[0,0] = 0.
    
    psi = ifft2(psi_fft)
    
    if t%Tplot == 0:
        plot_model(psi, zeta, t)